# Heart Stroke prediction - Machine Learning approach

In [38]:
import pandas as pd 
import numpy as np 
from ydata_profiling import ProfileReport

In [39]:
import matplotlib
matplotlib.use('TkAgg')

import matplotlib.pyplot as plt

In [40]:
# Set the display format for floating-point numbers
pd.set_option('display.float_format', '{:.4f}'.format)

## Data Preview

In [41]:
stroke_data = pd.read_csv('data/stroke_data.csv')

In [42]:
stroke_data

,id,age,avg_glucose_level,bmi,ever_married,feat01,feat02,feat03,feat04,feat05,...,feat08,feat09,feat10,gender,heart_disease,hypertension,Residence_type,smoking_status,stroke,work_type
0,1,43.0000,92.7100,30.5000,Yes,0.6216,0.6636,1.0703,1.2745,1.3907,...,1.1747,1.1307,0.4158,Male,0,0,Urban,formerly smoked,0,Private
1,2,59.0000,93.9000,42.2000,Yes,0.2858,0.4051,1.2722,0.9503,1.5604,...,0.9100,0.5848,0.5955,Male,0,0,Rural,never smoked,0,Private
2,3,25.0000,92.1400,36.2000,Yes,0.6959,0.4963,1.4668,1.0263,0.5139,...,1.5253,0.8223,0.1276,Male,0,0,Rural,Unknown,0,Private
3,4,74.0000,205.8400,54.6000,Yes,0.7181,0.4085,0.6438,0.8952,0.9654,...,1.4003,1.2426,0.3291,Female,0,1,Urban,never smoked,0,Self-employed
4,5,34.0000,79.8000,37.4000,Yes,0.4722,0.4638,1.1612,1.3084,0.8025,...,1.5124,1.3106,0.3367,Female,0,0,Urban,smokes,0,Private
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5354,5355,62.0000,98.0500,27.9000,Yes,0.6588,0.4631,1.1928,0.8352,1.0432,...,1.2921,0.7789,0.5882,Female,0,0,Rural,never smoked,0,Private
5355,5356,78.0000,244.9700,NaN,Yes,0.2496,0.4180,0.8447,0.7113,1.8057,...,0.7072,0.8729,0.4280,Male,0,0,Urban,formerly smoked,1,Private
5356,5357,56.0000,227.0400,23.0000,Yes,0.5955,0.6009,1.2593,0.2409,1.0876,...,1.3512,1.0232,0.7545,Female,0,0,Rural,smokes,0,Private
5357,5358,77.0000,190.6900,31.4000,Yes,0.3484,0.2541,0.9378,0.6646,1.4418,...,1.0762,1.1184,0.5017,Female,0,0,Rural,never smoked,1,Govt_job


In [43]:
stroke_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5359 entries, 0 to 5358
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5359 non-null   int64  
 1   age                5359 non-null   float64
 2   avg_glucose_level  5359 non-null   float64
 3   bmi                5118 non-null   float64
 4   ever_married       5359 non-null   object 
 5   feat01             5359 non-null   float64
 6   feat02             5359 non-null   float64
 7   feat03             5359 non-null   float64
 8   feat04             5359 non-null   float64
 9   feat05             5359 non-null   float64
 10  feat06             5359 non-null   float64
 11  feat07             5359 non-null   float64
 12  feat08             5359 non-null   float64
 13  feat09             5359 non-null   float64
 14  feat10             5359 non-null   float64
 15  gender             5359 non-null   object 
 16  heart_disease      5359 

In [44]:
stroke_data.columns

Index(['id', 'age', 'avg_glucose_level', 'bmi', 'ever_married', 'feat01',
       'feat02', 'feat03', 'feat04', 'feat05', 'feat06', 'feat07', 'feat08',
       'feat09', 'feat10', 'gender', 'heart_disease', 'hypertension',
       'Residence_type', 'smoking_status', 'stroke', 'work_type'],
      dtype='object')

We definitely do not want to include `id` field into the modelling part, so we remove it now.

In [45]:
stroke_data = stroke_data.drop(columns='id')

In [46]:
stroke_data = stroke_data.rename(columns={'Residence_type': 'residence_type'})

## Stratified train-test sampling

In [47]:
stroke_data.groupby('stroke')['stroke'].agg('count')/len(stroke_data)

stroke
0   0.9071
1   0.0929
Name: stroke, dtype: float64

In [48]:
# Perform stratified sampling with sklearn 
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(stroke_data, stroke_data['stroke']):
    train_set = stroke_data.loc[train_index]
    test_set = stroke_data.loc[test_index]

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(stroke_data.drop(columns = 'stroke'), stroke_data['stroke'], test_size=0.2, random_state=77)

In [50]:
y_train.value_counts()/len(y_train)

stroke
0   0.9067
1   0.0933
Name: count, dtype: float64

In [51]:
y_test.value_counts()/len(y_test)

stroke
0   0.9086
1   0.0914
Name: count, dtype: float64

## Missing values

In [52]:
# how many missings is there?
stroke_data.isna().sum()

age                    0
avg_glucose_level      0
bmi                  241
ever_married           0
feat01                 0
feat02                 0
feat03                 0
feat04                 0
feat05                 0
feat06                 0
feat07                 0
feat08                 0
feat09                 0
feat10                 0
gender                 0
heart_disease          0
hypertension           0
residence_type         0
smoking_status         0
stroke                 0
work_type              0
dtype: int64

In [53]:
from sklearn.impute import SimpleImputer

In [54]:
imputer =SimpleImputer(strategy="median")

In [55]:
imputer.fit(X_train.select_dtypes(exclude=['object'])[['bmi']])

SimpleImputer(strategy='median')

In [56]:
print(f'The median BMI in the train set is: {imputer.statistics_}')

The median BMI in the train set is: [28.1]


In [57]:
X_train[['bmi']] = imputer.transform(X_train[['bmi']])

In [58]:
X_test.isna().sum()

age                   0
avg_glucose_level     0
bmi                  43
ever_married          0
feat01                0
feat02                0
feat03                0
feat04                0
feat05                0
feat06                0
feat07                0
feat08                0
feat09                0
feat10                0
gender                0
heart_disease         0
hypertension          0
residence_type        0
smoking_status        0
work_type             0
dtype: int64

In [59]:
imputer.statistics_

array([28.1])

In [60]:
# the test set is imputed by the same number 
X_test[['bmi']] = imputer.transform(X_test[['bmi']])

## Exploratory Data Analysis

In [81]:
eda = ProfileReport(X_train, title='EDA Report', explorative=True)
eda.to_widgets()

Summarize dataset:  81%|████████▏ | 22/27 [00:00<00:00, 43.72it/s, Calculate auto correlation]        /Users/irenaz/.pyenv/versions/3.9.18/lib/python3.9/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: 'Urban'')
  warnings.warn(
Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

## Tree-based feature selection

### Factorization of categorical features

In [190]:
X_train_copy = X_train.copy()
cat_cols = X_train_copy.select_dtypes(include="object").columns

In [191]:
X_train_copy[cat_cols] = X_train_copy[cat_cols].apply(lambda x: pd.factorize(x)[0])

In [192]:
X_train_copy

,age,avg_glucose_level,bmi,ever_married,feat01,feat02,feat03,feat04,feat05,feat06,feat07,feat08,feat09,feat10,gender,heart_disease,hypertension,residence_type,smoking_status,work_type
1960,72.0000,85.8200,25.0000,0,0.5052,0.5884,0.6604,1.1819,1.1268,0.7556,0.4069,0.6632,1.0296,0.4590,0,1,0,0,0,0
1163,63.0000,93.2400,28.8000,0,0.3445,0.7257,1.2638,0.8641,1.1557,1.2759,1.2045,1.3485,0.9022,0.5199,1,0,0,1,1,1
3839,58.0000,170.9300,30.7000,1,0.4722,0.4665,1.4372,0.5551,1.5639,0.5440,0.6205,1.0394,0.8453,0.7252,0,0,0,1,2,0
3026,79.0000,90.7700,22.5000,0,0.2888,0.3997,1.1421,1.0072,0.6859,1.0088,0.6175,0.7305,0.3955,0.5396,1,0,0,1,1,1
5087,60.0000,86.0400,25.6000,0,0.6613,0.4053,1.1868,0.4444,1.1086,0.9252,1.0612,1.7483,0.9184,0.6597,0,0,1,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317,10.0000,69.2000,23.5000,1,0.3512,0.4463,1.4540,1.1966,1.3237,1.2983,0.6411,1.4213,0.4906,0.4946,0,0,0,1,0,3
2283,57.0000,106.2400,32.3000,0,0.6605,0.7062,0.6895,1.2730,1.1824,0.9115,1.0521,1.5312,1.7370,0.8475,0,0,0,0,1,1
2004,63.0000,62.1300,23.6000,0,0.5139,0.5608,1.2324,0.7516,1.6328,1.4094,0.9778,0.5758,0.7816,0.6385,1,0,1,0,1,1
3668,1.8000,95.2800,16.5000,1,0.6324,0.5525,0.6758,1.2759,0.7416,1.1128,0.8794,1.3113,0.7093,0.2258,1,0,0,1,2,3


In [193]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [194]:
forest = RandomForestClassifier(random_state=0)
forest.fit(X_train_copy, y_train)

RandomForestClassifier(random_state=0)

In [195]:
forest_importancies = pd.Series(forest.feature_importances_, index = X_train_copy.columns).sort_values(ascending=False)

In [196]:
forest_importancies

age                 0.1362
feat02              0.1252
avg_glucose_level   0.0792
feat10              0.0785
feat01              0.0773
feat06              0.0562
feat05              0.0529
bmi                 0.0523
feat04              0.0520
feat08              0.0513
feat09              0.0506
feat07              0.0504
feat03              0.0475
smoking_status      0.0211
hypertension        0.0162
work_type           0.0159
heart_disease       0.0117
ever_married        0.0094
residence_type      0.0082
gender              0.0081
dtype: float64

In [197]:
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)

In [198]:
fig, ax = plt.subplots()
forest_importancies.plot.bar(yerr = std, ax = ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()
plt.show()

### Factors as dummies

In [199]:
X_train_dummies = X_train.copy()
X_train_dummies = pd.get_dummies(X_train_dummies, drop_first=True)

In [200]:
forest = RandomForestClassifier(random_state=0)
forest.fit(X_train_dummies, y_train)

RandomForestClassifier(random_state=0)

In [201]:
forest_importancies = pd.Series(forest.feature_importances_, index = X_train_dummies.columns).sort_values(ascending=False)

In [202]:
forest_importancies

age                              0.1340
feat02                           0.1223
feat10                           0.0788
feat01                           0.0777
avg_glucose_level                0.0745
feat06                           0.0558
feat08                           0.0536
bmi                              0.0525
feat07                           0.0521
feat03                           0.0515
feat05                           0.0510
feat09                           0.0505
feat04                           0.0503
hypertension                     0.0162
heart_disease                    0.0119
ever_married_Yes                 0.0102
smoking_status_formerly smoked   0.0085
work_type_Private                0.0081
gender_Male                      0.0081
residence_type_Urban             0.0080
work_type_Self-employed          0.0079
smoking_status_never smoked      0.0075
smoking_status_smokes            0.0072
work_type_children               0.0018
work_type_Never_worked           0.0000


In [150]:
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
fig, ax = plt.subplots()
forest_importancies.plot.bar(yerr = std, ax = ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()
plt.show()

## One Hot Encoding & Scaling 

In [61]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer

In [62]:
num_columns = X_train.select_dtypes(exclude = 'object').columns.to_list()
cat_columns = X_train.select_dtypes(include = 'object').columns.to_list()

In [63]:
data_prep_pipeline = ColumnTransformer([
    ('num', StandardScaler(), num_columns), 
    ('cat', OneHotEncoder(sparse_output=False, drop='first'), cat_columns)
]).set_output(transform='pandas')

In [64]:
X_train = data_prep_pipeline.fit_transform(X_train)

In [65]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4287 entries, 1960 to 607
Data columns (total 26 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   num__age                             4287 non-null   float64
 1   num__avg_glucose_level               4287 non-null   float64
 2   num__bmi                             4287 non-null   float64
 3   num__feat01                          4287 non-null   float64
 4   num__feat02                          4287 non-null   float64
 5   num__feat03                          4287 non-null   float64
 6   num__feat04                          4287 non-null   float64
 7   num__feat05                          4287 non-null   float64
 8   num__feat06                          4287 non-null   float64
 9   num__feat07                          4287 non-null   float64
 10  num__feat08                          4287 non-null   float64
 11  num__feat09                      

In [66]:
X_train.isna().sum()

num__age                               0
num__avg_glucose_level                 0
num__bmi                               0
num__feat01                            0
num__feat02                            0
num__feat03                            0
num__feat04                            0
num__feat05                            0
num__feat06                            0
num__feat07                            0
num__feat08                            0
num__feat09                            0
num__feat10                            0
num__heart_disease                     0
num__hypertension                      0
cat__ever_married_Yes                  0
cat__gender_Male                       0
cat__gender_Other                      0
cat__residence_type_Urban              0
cat__smoking_status_formerly smoked    0
cat__smoking_status_never smoked       0
cat__smoking_status_smokes             0
cat__work_type_Never_worked            0
cat__work_type_Private                 0
cat__work_type_S

## Model Training

### Random Forest

In [71]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [72]:
# Define the parameter grid for RandomizedSearchCV
param_dist = {
    'n_estimators': [3,5,7,11],  # Number of trees in the forest
    'max_features': [2,4,6,8,26],     # Number of features to consider at each split
    'max_depth': [3, 5, 7]       # Maximum depth of the tree # Minimum number of samples required to be at a leaf node
}

# Initialize RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_classifier, 
                                   param_distributions=param_dist, 
                                   n_iter=50,  # Number of parameter settings that are sampled
                                   scoring='recall',  # Use recall as the scoring metric
                                   cv=5,  # Number of cross-validation folds
                                   n_jobs=-1,  # Use all available CPU cores
                                   random_state=42)

# Perform RandomizedSearchCV
random_search.fit(X_train, y_train)  # Assuming you have X_train and y_train for training data

# Get the best estimator from the search
best_rf_classifier = random_search.best_estimator_

# Now you can use best_rf_classifier for prediction or evaluation

In [77]:
best_rf_classifier.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 7,
 'max_features': 26,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 5,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

### AdaBoost 

In [67]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

model_adaboost = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth = 1), n_estimators=200, algorithm='SAMME.R', learning_rate=0.5

)

In [68]:
model_adaboost.fit(X_train, y_train)

/Users/irenaz/.pyenv/versions/3.9.18/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

## XGBoost